In [1]:
from datetime import date, timedelta

In [2]:
start_date = date.today() - timedelta(1)

In [3]:
bronze_adls = 'abfss://bronze@eqstorage.dfs.core.windows.net/'
silver_adls = 'abfss://silver@eqstorage.dfs.core.windows.net/'

In [2]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta


In [3]:
df = spark.read.option('multiline', 'true').json(f'{bronze_adls}{start_date}/earthquake_data.json')


NameError: name 'spark' is not defined

In [ ]:
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('magnitude'),
        col('properties.magType').alias('mag_type'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [ ]:
#Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [ ]:
#Save the transformed dataframe to silver container
silver_output_path = f'{silver_adls}earthquake_events_silver/'


In [ ]:
#Append Dataframe to silver container in parquet format
df.write.mode('append').parquet(silver_output_path)